In [2]:
import os 

In [3]:
%pwd

'/workspaces/End-End-Text-summeriser/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/workspaces/End-End-Text-summeriser'

In [6]:
#  3. entity - define the return type of the function

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen  = True) # define the custom return type of the function - decorator function
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [7]:
# 4. configuration manager in src config.
# create constant in the constant folder->__init__py file

from textSummerizer.constants import *
from textSummerizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    #constructor assigning values from constant file to the class variables
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        # reading yaml config file and params file and assigning them to the class variables

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # create directories/file for artifacts
        # note that we are using . to access, as we have defined using configbox

        create_directories([self.config.artifacts_root])

    # user defined functions with parameters returned type are based on the decorator function defined in above cell

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [4]:
#5. components

In [9]:
import os
import urllib.request as request # download and unzip files
import zipfile
from textSummerizer.logging import logger # custom logger
from textSummerizer.utils.common import get_size # get size module from utils.common

In [10]:
# create class for data ingestion component will use the class configuration manager
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 
        # assign config to the class variable and intialize instance variable


    # download and unzip files from source URL(defined  in the config.yaml) to local data file 
    # logging the download progress in the logger and return the size of the downloaded file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    # unzip files from local data file to unzip_dir
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
# 6. pipeline

In [11]:
try:
    config = ConfigurationManager() # initialize the class configuration manager
    data_ingestion_config = config.get_data_ingestion_config() # assign config to the class variable
    data_ingestion = DataIngestion(config=data_ingestion_config) # initialize the class data ingestion component
    data_ingestion.download_file() # download and unzip files from source URL(defined  in the config.yaml) to local data
    data_ingestion.extract_zip_file()  # unzip files from local data file to unzip_dir
except Exception as e:
    raise e

[2024-03-19 03:08:45,984: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-19 03:08:45,999: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-19 03:08:46,000: INFO: common: created directory at: artifacts]
[2024-03-19 03:08:46,001: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-19 03:08:46,003: INFO: 663868664: File already exists of size: ~ 7718 KB]


In [9]:
# change the working directory back to the original root directory before excuting the code.
%pwd

'/workspaces/End-End-Text-summeriser/research'

In [ ]:
# data ingestion component working fine and now will be updated in the modules for creating the components in the src folder.
# artifacts will be created in the artifacts directory
# before committing the artifacts/* will be added in the .gitignore file.